In [27]:
import torch
import torch.nn as nn

net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,2))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.1462, -0.3723],
        [-0.1807, -0.4056]], grad_fn=<AddmmBackward0>)

In [5]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3288,  0.1821, -0.0848,  0.3490,  0.1788,  0.1862,  0.2969, -0.0387],
        [-0.2160,  0.1034,  0.0099, -0.2802, -0.3361,  0.1338, -0.0258, -0.2785]])), ('bias', tensor([-0.0126, -0.1279]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0126, -0.1279], requires_grad=True)
tensor([-0.0126, -0.1279])
None


In [14]:
# 第一句：获取网络第0层（第一个线性层）的参数名称和形状
# net[0].named_parameters() 返回一个迭代器，包含 (参数名, 参数张量) 的元组
# [(name,param.shape) for name,param in net[0].named_parameters()] 是列表推导式，创建 (名称, 形状) 的元组列表
# *[] 是解包操作符，将列表中的元素作为单独的参数传递给print函数
print(*[(name,param.shape) for name,param in net[0].named_parameters()])

# 第二句：获取整个网络所有层的参数名称和形状
# net.named_parameters() 返回整个网络所有参数的迭代器
# 同样使用列表推导式和解包操作符来打印所有参数的名称和形状
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([2, 8])) ('2.bias', torch.Size([2]))


In [15]:
net.state_dict()['0.weight']

tensor([[ 0.2343,  0.2919,  0.0285, -0.1826],
        [-0.3815,  0.2763,  0.4414, -0.0115],
        [-0.2691,  0.1722, -0.3278,  0.0622],
        [-0.1397,  0.4487, -0.4331, -0.4935],
        [-0.2547, -0.3323,  0.4475, -0.3757],
        [ 0.4219, -0.1702,  0.3314,  0.2822],
        [-0.3926,  0.0924, -0.4880, -0.3910],
        [-0.0962,  0.2078, -0.0758,  0.2098]])

In [19]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())
def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
net=nn.Sequential(block2(),nn.Linear(4,1))
X=torch.rand(size=(2,4))
net(X)
print(net)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [20]:
print(net[0][1][0].bias.data)

tensor([ 0.1387, -0.2488, -0.2691,  0.4893,  0.2780, -0.0059, -0.3875, -0.1251])


In [28]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0068,  0.0018,  0.0082, -0.0033]), tensor(0.))

In [35]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias,2)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(2.))

In [30]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.1961, -0.1797, -0.6635, -0.1239])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.],
        [42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        # 这句话的意思是：将权重矩阵中绝对值小于5的元素设置为0，绝对值大于等于5的元素保持不变
        # m.weight.data.abs() >= 5 会创建一个布尔掩码，True表示绝对值>=5，False表示绝对值<5
        # 与权重数据相乘后，True在PyTorch中会被转换为1，乘以1保持原值不变；False会被转换为0，乘以0变为0
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

with torch.no_grad():
    net[0].weight[0,0]=100
print(net[0].weight.data[0])

In [36]:
#参数共享？用同一个变量名即可
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
